Для описания стандартных архитектур, в Keras уже сущуствуют классы для слоев и моделей

Dense () - полносвязный слой \
Conv1D, Conv2D, Conv3D  - сверточные слои \
Conv2DTranspose, Conv3DTranspose - транспонированные (обратные) сверточные слои \
SimpleRNN, LSTM, GRU - рекуррентные слои \
MaxPooling2D, Dropout, BatchNormalization  - вспомогательные слои \

https://keras.io/api/layers/

Model - общий класс модели \
Sequential - последовательная модель

#### Общие функции

In [ ]:
model.summary() # показывает информацию о модели
model.add(Dense(5,activation = 'linear', name = 'layer') # добавляет слой к модели
model.layers # показывает слои
model.layers[0].weights  # показывает веса слоя


In [11]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow import keras

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [69]:
model = keras.Sequential([
#    Input(shape = (20, )),   Это значит что тензор на вход должен быть [batch_size,20]
    Dense(128, activation = 'relu', input_shape = (784,), name = 'hidden'),
    Dense(10, activation = 'softmax', name = 'output_layer')
])

model_ex = keras.Model(inputs = model.inputs,
                        outputs = [layer.output for layer in model.layers]) # добавим еще одну модель на основе 1-ой, которая её не меняет но, выходами дает и с последнего слоя и со скрытого

# Размертность модели устанавливается при запуске входного тензора, если не прописано явно. до этого веса у нее нулевые 
x = tf.random.uniform((1,784),0,1)
model(x)

In [71]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train/255
x_test = x_test/255

x_train = tf.reshape(tf.cast(x_train,tf.float32), [-1,28*28])
x_test = tf.reshape(tf.cast(x_test,tf.float32), [-1,28*28])

y_train = to_categorical(y_train,10)
y_test_cat = to_categorical(y_test,10)

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size =32, epochs =5)
print (model.evaluate(x_test,y_test_cat))


x = tf.expand_dims(x_test[0],axis =0)
y = model_ex(x)
y2 = model(x)
print (y,y2, sep='\n\n')

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2590 - accuracy: 0.9259
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1138 - accuracy: 0.9660
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0772 - accuracy: 0.9765
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0579 - accuracy: 0.9829
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0857 - accuracy: 0.9745
[0.08570753037929535, 0.9745000004768372]


#### Можно легко добавлять слои и дообучать

In [72]:
model_ex_2= keras.Sequential([model,
                             Dense(10,activation ='softmax')
                             ])
#model.trainable = False # чтобы не обучалась заново model
model.layers[0].trainable = False #Можно чтобы не переобучалася только первый слой
model_ex_2.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

model_ex_2.fit(x_train,y_train,batch_size =32, epochs =3)
print (model_ex_2.evaluate(x_test,y_test_cat))

Epoch 1/3
1875/1875 [==============================] - 6s 3ms/step - loss: 1.1145 - accuracy: 0.8547
Epoch 2/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3541 - accuracy: 0.8979
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.2554 - accuracy: 0.8845
[0.2554381489753723, 0.8845000267028809]


### _________________________________________________________________________

In [14]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers

In [7]:
tf.random.set_seed(1)

In [17]:
# Опишем сеть не используя класс Sequential
# Это пока не модель, а просто сеть, обучать нельзя
input = keras.Input(shape=(32,32,3))
x = layers.Conv2D(32,3, activation = 'relu')(input)
x = layers.MaxPooling2D(2, padding = 'same')(x)
x = layers.Conv2D(64,3, activation = 'relu')(x)
x = layers.MaxPooling2D(2, padding = 'same')(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(10,activation = 'softmax')(x)

#А вот теперь уже модель
model = keras.Model(inputs=input, outputs = output)

In [18]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 3136)              0   

In [29]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [30]:
x_train = x_train/255
x_test = x_test/255

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [31]:
model.fit(x_train,y_train, batch_size = 64, epochs =20, validation_split = 0.2)

Epoch 1/20
625/625 [==============================] - 49s 74ms/step - loss: 1.5825 - accuracy: 0.4237 - val_loss: 1.2767 - val_accuracy: 0.5540
Epoch 2/20
625/625 [==============================] - 46s 73ms/step - loss: 1.2175 - accuracy: 0.5671 - val_loss: 1.0853 - val_accuracy: 0.6239
Epoch 3/20
625/625 [==============================] - 46s 73ms/step - loss: 1.0745 - accuracy: 0.6227 - val_loss: 1.0048 - val_accuracy: 0.6530
Epoch 4/20
625/625 [==============================] - 51s 81ms/step - loss: 0.9781 - accuracy: 0.6562 - val_loss: 0.9325 - val_accuracy: 0.6769
Epoch 5/20
625/625 [==============================] - 46s 74ms/step - loss: 0.9135 - accuracy: 0.6784 - val_loss: 0.8918 - val_accuracy: 0.6908
Epoch 6/20
625/625 [==============================] - 53s 85ms/step - loss: 0.8591 - accuracy: 0.6979 - val_loss: 0.8748 - val_accuracy: 0.6977
Epoch 7/20
625/625 [==============================] - 55s 87ms/step - loss: 0.8037 - accuracy: 0.7166 - val_loss: 0.8509 - val_accuracy:

In [33]:
import numpy as np


array([[1, 2, 3],
       [1, 2, 3]])